Veri işleme ve analizi için pandas kütüphanesi kullanılacaktır. Pandas kütüphanesi pd rumuzu ile kısaltılmıştır.

In [1]:
import pandas as pd

Verilerimiz Google Drive (bulut) üzerinde olduğu için Google Colab ile Google Drive arasında bağlantı kurulacaktır.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 isimli klasörde verilerimizi sakladığımız için bu klasöre erişilecektir.

***NOT:*** Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!

In [ ]:
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
# Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 diye bir klasör oluşturduktan sonra o klasöre erişiyoruz.
# Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!
%cd TDDI2022/

# Klasörün içeriğinin görülmesi için
%ls

**Google Colab Pro** Hizmetlerinden yararlanarak hem daha iyi Grafik İşlem Birimine (GPU) hem de daha yüksek bellek birimine (RAM) erişerek büyük veriyi işleyeceğiz. Google Colab Pro hesabını verimli kullanmak için sadece gerektiğinde **'Çalışma zamanı'** menüsünden **'Çalışma zamanı türünü değiştir'** diyerek GPU/TPU ve Yüksek RAM seçeneklerini kullanacağız.

In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU aracı bulunamadı!') # Gereksiz kullanım olmaması ve Google Colab PRO hesabımızı verimli kullanmak için şimdilik sadece yüksek RAM'de çalışıyoruz.
else:
  print('GPU bulundu: {}'.format(device_name))

GPU aracı bulunamadı!


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Bir GPU\'ya bağlanılamadı!')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Çalışma zamanınız {:.1f} gigabytes\'lık RAM\'a sahip\n'.format(ram_gb))

if ram_gb < 20:
  print('Yüksek bir RAM Çalışma Zamanı KULLANMIYORSUNUZ!')
else:
  print('Yüksek bir RAM Çalışma Zamanı Kullanıyorsunuz!')

Bir GPU'ya bağlanılamadı!
Çalışma zamanınız 37.8 gigabytes'lık RAM'a sahip

Yüksek bir RAM Çalışma Zamanı Kullanıyorsunuz!


TDDI2022 klasöründe bulunan ve proje kapsamında oluşturulan **filmOzetleriVeriKumesi_TURKCE.csv** dosyasının içindeki veriler **veri İskeleti** oluşturmak için pandas ile okunacaktır.

Veri iskeleti oluşturulduktan sonra veri iskeleti ile ilgili bilgi ekrana yazdırılacaktır. Tam özetin yazdırılıp yazdırılmayacağı durumunu *verbose=True* (yazdırılsın) parametresi ile kontrol edilecektir. 

In [6]:
df = pd.read_csv('filmOzetleriVeriKumesi_TURKCE.csv')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      34886 non-null  int64 
 1   Çıkış Yılı      34886 non-null  int64 
 2   Orijinal İsmi   34886 non-null  object
 3   Yapıldığı Ülke  34886 non-null  object
 4   Yönetmeni       34886 non-null  object
 5   Oyuncu Kadrosu  33464 non-null  object
 6   Türü            34858 non-null  object
 7   Veri Kaynağı    34886 non-null  object
 8   Olay Dizisi     34886 non-null  object
dtypes: int64(2), object(7)
memory usage: 2.4+ MB


Unnamed: 0 isimli sütun / alan proje kapsamında kullanılması zorunlu olan veri içermediği için silinecektir.

In [7]:
df = df.drop(columns='Unnamed: 0')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Çıkış Yılı      34886 non-null  int64 
 1   Orijinal İsmi   34886 non-null  object
 2   Yapıldığı Ülke  34886 non-null  object
 3   Yönetmeni       34886 non-null  object
 4   Oyuncu Kadrosu  33464 non-null  object
 5   Türü            34858 non-null  object
 6   Veri Kaynağı    34886 non-null  object
 7   Olay Dizisi     34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


Bellek ve işlem gücünden tasarruf etmek için df['Olay Dizisi'] alanında veri ön işlemleri yapmamız gerekiyor.

**Kaynaklar:**
* https://practicaldatascience.co.uk/machine-learning/how-to-preprocess-text-for-nlp-in-four-easy-steps

Film ararken işe yaramayacak \n \r \\ veya benzer karakterler veri iskeletinden temizlenecektir. 

In [8]:
df = df.replace('  ','')
df = df.replace(' ','')
df = df.replace('.','')
df = df.replace(':','')
df = df.replace('\\','')
df = df.replace('\n','')
df = df.replace('\r','')
df = df.replace('i̇','i')
df = df.replace('ý','ı')

Özellikle olay dizisi orijinal kaynağı wiki olduğu için atıflardan kaynaklı [1] ve [2] gibi girdilere sahiptir. Bunları da temizlememiz gerekmektedir.

In [9]:
df['Olay Dizisi'] = df['Olay Dizisi'].replace('1','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('2','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('3','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('4','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('5','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('6','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('7','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('8','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('9','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace('[','')
df['Olay Dizisi'] = df['Olay Dizisi'].replace(']','')

Özetten arama yapmak için istediğimiz özelliklerin olduğu bir film girdisi yapılacaktır.

Örnek girdi "çalışmak için gökyüzünde ev yapan insanın herkesle iyi geçinmesi" olsun ve bu girdiye göre elimizdeki film listesinden film önerisi yapmaya çalışalım. Ayrıca büyük küçük harf ile ilgili farklılıklar yaşamamak için her zaman küçük harf olarak çalışalım.

In [63]:
arananFilmOzeti = 'çalışmak için gökyüzünde ev yapan insanın herkesle iyi geçinmesi'.lower()

Aramayı yapmadan önce arananFilmOzeti içinde gereksiz kelime bırakmayalım. Simgeleştirme (tokenize) için de Türkçe'ye uyumlu bir kelime simgeleştirici (word tokenizer) kullanalım.

**Kaynaklar:** 

* https://www.veribilimiokulu.com/natural-language-toolkitnltk/
* https://medium.com/kodcular/do%C4%9Fal-dil-i%CC%87%C5%9Fleme-nedir-uygulamal%C4%B1-%C3%B6rnek-b8eb15af7fd5
* https://melikebektas95.medium.com/zemberek-k%C3%BCt%C3%BCphanesi-ile-t%C3%BCrk%C3%A7e-metinlerde-kelime-k%C3%B6klerinin-bulunmas%C4%B1-6ddd3a875d5f

Türkçe öz bulma için Zemberek'i kurup deneme yapıyoruz.

**Kaynaklar:**
* https://github.com/otuncelli/turkish-stemmer-python
* https://snowballstem.org/algorithms/turkish/stemmer.html
* https://www.nltk.org/howto/stem.html

In [11]:
!pip install zemberek-python

from zemberek import TurkishMorphology

ozBulucu = TurkishMorphology.create_with_defaults()

kelimeler = ['kalem','ilişkilendiremediklerimiz','gözlük',' gözlem']

for kelime in kelimeler:
  kelime = str(ozBulucu.analyze(kelime))
  print(kelime)
  print(kelime[kelime.find('analysisResults=[')+17:kelime.find(':')])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93.6 MB 1.7 MB/s 
     |████████████████████████████████| 112 kB 61.9 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=82a16dfd61583729e179647d3ed194479ffafcafa4c3ebe95f51a7631099199d
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 9.88522458076477


2022-08-18 09:11:59,328 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 9.88522458076477

WordAnalysis{input='kalem', normalizedInput='kalem', analysisResults=[Kale:Noun, Prop] kale:Noun+A3sg+m:P1sg [kale:Noun] kale:Noun+A3sg+m:P1sg [kalem:Noun] kalem:Noun+A3sg}
Kale
WordAnalysis{input='ilişkilendiremediklerimiz', normalizedInput='ilişkilendiremediklerimiz', analysisResults=[ilişki:Noun] ilişki:Noun+A3sg|len:Acquire→Verb|dir:Caus→Verb+eme:Unable|dik:PastPart→Noun+ler:A3pl+imiz:P1pl}
ilişki
WordAnalysis{input='gözlük', normalizedInput='gözlük', analysisResults=[gözlük:Noun] gözlük:Noun+A3sg [göz:Noun] göz:Noun+A3sg|lük:Ness→Noun+A3sg}
gözlük
WordAnalysis{input='gözlem', normalizedInput='gözlem', analysisResults=[gözlem:Noun] gözlem:Noun+A3sg [Gözlem:Noun, Prop] gözlem:Noun+A3sg}
gözlem


Zemberek'in sonuçları bizim için çok iyi ama maliyet etkinliği zaman açısından çok pahalı geldi. Türkçe'ye uyumlu diğer öz / kök bulucu araçları deniyoruz.

**Kaynaklar:**
* https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
* https://www.tutorialspoint.com/natural_language_toolkit/natural_language_toolkit_stemming_lemmatization.htm

In [12]:
testEdilecekKelimeler = ['kalem','ilişkilendiremediklerimiz','gözlük','gözlem']
print('Test edilecek kelimeler:',testEdilecekKelimeler)
print('Kök Bulma YÖNTEM 1 - snowballstemmer','\n',50*'-',sep='')
!pip install snowballstemmer
from snowballstemmer import TurkishStemmer
turkStem = TurkishStemmer()
testSonucu = []
for kelime in testEdilecekKelimeler:
  testSonucu.append(turkStem.stemWord(kelime))
print(testSonucu,'\n\n\n')

print('Kök Bulma YÖNTEM 2 - nltk','\n',50*'-',sep='')
!pip install nltk
from nltk.stem import PorterStemmer
ps=PorterStemmer()
testSonucu = []
for kelime in testEdilecekKelimeler:
  testSonucu.append(ps.stem(kelime))
print(testSonucu,'\n\n\n')

print('Kök Bulma YÖNTEM 3 - TurkishStemmer','\n',50*'-',sep='')
!pip install TurkishStemmer
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()
testSonucu = []
for kelime in testEdilecekKelimeler:
  testSonucu.append(stemmer.stem(kelime))
print(testSonucu,'\n\n\n')

print('Kök Bulma YÖNTEM 4 - WordNetLemmatizer','\n',50*'-',sep='')
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
testSonucu = []
for kelime in testEdilecekKelimeler:
  testSonucu.append(lemmatizer.lemmatize(kelime))
print(testSonucu,'\n\n\n')

print('Kök Bulma YÖNTEM 5 - LancasterStemmer','\n',50*'-',sep='')
import nltk
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()
testSonucu = []
for kelime in testEdilecekKelimeler:
  testSonucu.append(stemmer.stem(kelime))
print(testSonucu,'\n\n\n')

print('Kök Bulma YÖNTEM 6 - RegexpStemmer','\n',50*'-',sep='')
import nltk
from nltk.stem import RegexpStemmer
stemmer = RegexpStemmer('tr')
testSonucu = []
for kelime in testEdilecekKelimeler:
  testSonucu.append(stemmer.stem(kelime))
print(testSonucu,'\n\n\n')




Test edilecek kelimeler: ['kalem', 'ilişkilendiremediklerimiz', 'gözlük', 'gözlem']
Kök Bulma YÖNTEM 1 - snowballstemmer
--------------------------------------------------
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['kale', 'ilişkilendiremedik', 'gözlük', 'gözle'] 



Kök Bulma YÖNTEM 2 - nltk
--------------------------------------------------
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['kalem', 'ilişkilendiremediklerimiz', 'gözlük', 'gözlem'] 



Kök Bulma YÖNTEM 3 - TurkishStemmer
--------------------------------------------------
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['kalem', 'ilişkilendiremedik', 'gözlük', 'gözle'] 



Kök Bulma YÖNTEM 4 - WordNetLemmatizer
--------------------------------------------------


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


['kalem', 'ilişkilendiremediklerimiz', 'gözlük', 'gözlem'] 



Kök Bulma YÖNTEM 5 - LancasterStemmer
--------------------------------------------------
['kalem', 'ilişkilendiremediklerim', 'gözlük', 'gözlem'] 



Kök Bulma YÖNTEM 6 - RegexpStemmer
--------------------------------------------------
['kalem', 'ilişkilendiremediklerimiz', 'gözlük', 'gözlem'] 





Alternatif Kök Bulma Yöntemlerinin "ilişkilendiremediklerimiz" kelimesini ZEMBEREK'in yaptığı gibi "ilişki" olarak özüne indirememesinden ötürü ZEMBEREK ile devam etmeyi seçtik.
Gereksiz kelimeleri çıkartıyoruz. (Tokenizer olarak Türkçe'ye daha uyumlu olan TrTokenizer kullanıyoruz)

**Kaynaklar:**
* https://github.com/apdullahyayik/TrTokenizer
* https://pythonprogramming.net/stop-words-nltk-tutorial/

In [13]:
!pip install trtokenizer
from trtokenizer.tr_tokenizer import SentenceTokenizer, WordTokenizer

cumleSimgelestirmeNesnesi = SentenceTokenizer()
kelimeSimgelestirmeNesnesi = WordTokenizer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
# Gereksiz kelimeleri çıkartıyoruz
import nltk
nltk.download('stopwords') # Gereksiz Kelimeler
nltk.download('punkt')     # Noktalama İşaretleri
from nltk.corpus import stopwords

gereksizler = set(stopwords.words('turkish'))
cumle = ' '
kelimeler = kelimeSimgelestirmeNesnesi.tokenize(cumle.join(cumleSimgelestirmeNesnesi.tokenize(arananFilmOzeti)))
gereksizKelimelerCikarilmisArananFilmOzeti = []
 
for k in kelimeler:
    if k not in gereksizler:
        gereksizKelimelerCikarilmisArananFilmOzeti.append(k)
 
print(gereksizKelimelerCikarilmisArananFilmOzeti)

['çalışmak', 'gökyüzünde', 'ev', 'yapan', 'insanın', 'herkesle', 'iyi', 'geçinmesi']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Veri Kümemizde de işlemler yapmamız gerekiyor. Önce simgeleştirme yapmak için bir fonksiyon yazıp veri iskeletinde 'olay dizisi' sütununda yer alan film özetlerimizi kelime kelime ayırıyoruz.

In [15]:
def simgelestir(sutun):
  simgelesitirilmisSutundakiler = kelimeSimgelestirmeNesnesi.tokenize(sutun)
  return list(set([w.lower() for w in simgelesitirilmisSutundakiler if w.isalpha()]))

In [16]:
df['simgelestirilmis'] = df.apply(lambda x: simgelestir(x['Olay Dizisi']), axis=1)
df[['Orijinal İsmi', 'simgelestirilmis']].head(5)

,Orijinal İsmi,simgelestirilmis
0,Kansas Saloon Smashers,"[daha, çıkmadan, müşterilere, nation, carrie, ..."
1,Love by the Light of the Moon,"[yukarı, daha, gökyüzünü, neden, ve, kadının, ..."
2,The Martyred Presidents,"[daha, uzun, portalı, ve, üç, garfield, sekiz,..."
3,"Terrible Teddy, the Grizzly King","[çekimin, kadar, çekime, eden, temsil, başkan,..."
4,Jack and the Beanstalk,"[yukarı, onu, daha, kıl, kovalar, kadar, zorla..."


Veri Kümemizde işlemlere devam ediyoruz. Simgeleştirme yaptıktan sonra gereksiz kelimeleri ayıklamak için bir fonksiyon yazıp veri iskeletinde 'olay dizisi' sütununda yer alan film özetlerimizi kelime kelime ayıklıyoruz.

In [17]:
import nltk
nltk.download('stopwords')
def remove_stopwords(simgelestirilmisSutun):
  from nltk.corpus import stopwords
  stops = set(stopwords.words("turkish"))
  return [word for word in simgelestirilmisSutun if not word in stops]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
df['sadelestirilmis'] = df.apply(lambda x: remove_stopwords(x['simgelestirilmis']), axis=1)
df[['Orijinal İsmi', 'sadelestirilmis']].head()

,Orijinal İsmi,sadelestirilmis
0,Kansas Saloon Smashers,"[çıkmadan, müşterilere, nation, carrie, salond..."
1,Love by the Light of the Moon,"[yukarı, gökyüzünü, kadının, geçen, şeyi, omzu..."
2,The Martyred Presidents,"[uzun, portalı, üç, garfield, sekiz, olan, yer..."
3,"Terrible Teddy, the Grizzly King","[çekimin, kadar, çekime, eden, temsil, başkan,..."
4,Jack and the Beanstalk,"[yukarı, onu, kıl, kovalar, kadar, zorla, sapı..."


Veri ön / hazırlık işlemlerimize devam ediyoruz. Kelimelerin özünü / kökünü bulmak için için bir fonksiyon yazıp veri iskeletinde 'olay dizisi' sütununda yer alan film özetlerimizin simgeleştirilmiş ve ayıklanmış kelimelerinin özünü buluyoruz.

In [39]:
from zemberek import TurkishMorphology
ozBulucu = TurkishMorphology.create_with_defaults()

def ozunuBuldur(simgelestirilmisSutun):
  # Zemberek ile deniyoruz
  sonucListesi = []
  for kelime in simgelestirilmisSutun:
    kelime = str(ozBulucu.analyze(kelime)).lower()
    kelime = kelime[kelime.find('analysisresults=[')+17:kelime.find(':')]
    sonucListesi.append(kelime)
  return sonucListesi

INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 12.305127143859863


2022-08-18 09:49:48,325 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 12.305127143859863



Basit bir girdi ile fonksiyonumuzu test ediyoruz.

In [20]:
# print(ozunuBuldur(gereksizKelimelerCikarilmisArananFilmOzeti))
print(arananFilmOzeti)

adamın biri gece salonda kitap okuyormuş ve hayal dünyasına dalmış


Türkçe özünü bulma (stemmig) işlemi Google Colab Standart hesabı ile yaklaşık 6 dakika sürüyor. Google Colab Pro hesabı ile yüksek RAM (35.24 GB) ve TPU kullanarak süre 5 dakikaya düşüyor.

**ÖNEMLİ BULGU:**

NLTK Stemming Tool ile TurkishStemmer arasında 100000 civarında kelime oynuyor.

Yine daha büyük ama basit bir girdiyle daha fonksiyonumuzu test ediyoruz.

In [40]:
print(df['sadelestirilmis'][0])
print(ozunuBuldur(df['sadelestirilmis'][0]))

['çıkmadan', 'müşterilere', 'nation', 'carrie', 'salonda', 'aynaları', 'takipçileri', 'kafasına', 'gitmesini', 'birayı', 'saldırdılar', 'herkesin', 'barmen', 'yüzüne', 'barı', 'içki', 'demirbaşları', 'kasayı', 'adama', 'püskürtür', 'grup', 'içeri', 'ardından', 'maden', 'sonra', 'adamın', 'ortaya', 'kovasını', 'mahvetmeye', 'önce', 'kırmaya', 'çektiler', 'yazar', 'şapkasını', 'suyu', 'çalışıyor', 'i̇rlandalı', 'basmakalıp', 'doldurduktan', 'gözlerine', 'bir', 'birayla', 'servisi', 'polis']
['çıkmak', 'müşteri', 'nation', 'carrie', 'salon', 'ayna', 'takip', 'kafa', 'gitmek', 'bira', 'saldırmak', 'herkes', 'barmen', 'yüz', 'bar', 'içki', 'demirbaş', 'kasa', 'adamak', 'püskürmek', 'grup', 'içeri', 'ardından', 'maden', 'sonra', 'ada', 'orta', 'kova', 'mahvetmek', 'önce', 'kırmak', 'çekmek', 'yazar', 'şapka', 'su', 'çalışmak', "ut='i̇rlandalı', normalizedinput='i̇rlandalı', analysisresults=", 'basmakalıp', 'doldurmak', 'göz', 'bir', 'bira', 'servi', 'polis']


Yeterince test yaptığımızı düşünerek ve fonksiyonumuzun istediğimiz gibi çalıştığından emin olarak veri kümemizdeki son ön / hazırlık işlemimize başlıyoruz. Veri iskeletinde 'olay dizisi' sütununda yer alan film özetlerimizin içindeki kelimelerin özünü buluyoruz. Başka bir değişle, limonatanın şekerini, suyunu ve nanesini alıp saf limon haline dönüştürüyoruz. 

**MOTTO:** Biz limonataya limonata demeyiz içindeki limon tadı belli olmayınca ;)

In [41]:
df['ozunuBulmus'] = df.apply(lambda x: ozunuBuldur(x['sadelestirilmis']), axis=1)
df[['Orijinal İsmi', 'ozunuBulmus']].head()

,Orijinal İsmi,ozunuBulmus
0,Kansas Saloon Smashers,"[çıkmak, müşteri, nation, carrie, salon, ayna,..."
1,Love by the Light of the Moon,"[yukarı, gökyüzü, kadı, geçen, şe, omuz, gülüm..."
2,The Martyred Presidents,"[uzun, ut='portalı', normalizedinput='portalı'..."
3,"Terrible Teddy, the Grizzly King","[çek, kadar, çek, etmek, temsil, başkan, ev, t..."
4,Jack and the Beanstalk,"[yukarı, o, kıl, kovmak, kadar, zorla, sap, in..."


ZEMBEREK fonksiyonumuzda kural bazlıayıklama sistemimizin 3. satırdaki **portal** olarak bulması gereken yere **ut='portalı', normalizedinput='portalı', analysisresults=,** verisini yazdığını görünce kuralımızda istisna yakaladığımızı fark ediyor ama yarışma zamanımıza ve ZEMBEREK'in işlem süresinin 3 saat aldığına bakarak mevcut hali ile devam ediyoruz.

AMA yapabileceğimiz akıllıca dokunuş ile istisnayı dışarıdan değiştirmemizin çaresini pas geçmiyoruz.

In [99]:
def istisnalariDuzelt(istisnaliVeri):
  sonucListesi = []
  kelimeler = str(istisnaliVeri.lower())
  kelimeler = kelimeler.split(',')
  for kelime in kelimeler:
    if 'ut=' in kelime:
      sonucListesi.append(kelime[3:-1])
    elif "'" in kelime:
      sonucListesi.append(kelime[15:-1])
    elif 'normalizedinput=' in kelime:
      sonucListesi.append(kelime[17:-1])
    elif 'nput=' in kelime:
      sonucListesi.append(kelime[18:-1])
    elif 'analysisresults=' in kelime:
      pass
    else:
      sonucListesi.append(kelime)
  sonucListesi = list(set(sonucListesi))
  return sonucListesi

İstisnai durumları basit bir girdi ile test ediyoruz.

In [100]:
print(istisnalariDuzelt("uzun,ut='portalı', normalizedinput='portalı', analysisresults=,üç,garfield,sekiz,olmak,yermek,saniye,sermek,iki,suikast,james,ayak,dip,william,çek,lincoln,oturmak,portre,biraz,başkan,mezar,suikast,sürmek,ut='i̇lkinde', normalizedinput='i̇lkinde', analysisresults=,kız,orta,ikinci,sunak,adalet,kurban,abd,atmak,görüntülemek,dakika,kamera,bir,yüz,film"))

['sermek', 'mezar', 'uzun', "rmalizedinput='i̇lkinde", "rmalizedinput='portalı", 'sunak', "'portalı", 'üç', 'yermek', 'garfield', 'sekiz', 'dip', 'iki', 'olmak', 'ayak', 'başkan', 'suikast', 'atmak', 'james', 'dakika', 'çek', 'william', 'orta', 'saniye', 'görüntülemek', 'yüz', 'biraz', 'sürmek', "'i̇lkinde", 'kız', 'ikinci', 'oturmak', 'adalet', 'kurban', 'abd', 'portre', 'bir', 'lincoln', 'film', 'kamera']


Hala kabuledilebilir düzeyde hata / istisna olduğunu görüyoruz ama artık ilerliyoruz ve 'ozunuBulmus' alanındaki istisnaları kabuledilebilir hata bırakacak şekilde düzeltiyoruz.

In [101]:
# Jupiter Notebook çalışma sırasından dolayı 'Ozet' alanını yazdık ama yukarıdan aşağı çalıştırılırken 'ozunuBulmus'
df['Ozet2'] = df.apply(lambda x: istisnalariDuzelt(x['Ozet']), axis=1)
df[['Orijinal İsmi', 'Ozet2']].head()

,Orijinal İsmi,Ozet2
0,Kansas Saloon Smashers,"[rmalizedinput='i̇rlandalı, mek herkes nation ..."
1,Love by the Light of the Moon,[yukarı korku engellemek geçen kadı terk görüş...
2,The Martyred Presidents,"[rmalizedinput='i̇lkinde, rmalizedinput='porta..."
3,"Terrible Teddy, the Grizzly King","[rmalizedinput='i̇kinci, i yardımcı tepe kadar..."
4,Jack and the Beanstalk,[yukarı dönmek kıl sırık tepe bırakmak kadar z...


In [ ]:
df.to_csv('ozet2.csv',columns=['Ozet2'])
df[['Orijinal İsmi', 'Ozet2']].head()

In [111]:
df2 = pd.read_csv('ozet2.csv')
df2.head()
df = df.drop(columns=['Ozet'])
df['Ozet'] = df2['Ozet2']

In [103]:
def kelimeleriBirlestir(simgelestirilmisSutun):
  simgelestirilmisSutun = list(set(simgelestirilmisSutun))
  return (' '.join(simgelestirilmisSutun))

In [ ]:
df['Ozet'] = df.apply(lambda x: kelimeleriBirlestir(x['ozunuBulmus']), axis=1)
df[['Orijinal İsmi', 'Ozet']].head()

In [112]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Çıkış Yılı      34886 non-null  int64 
 1   Orijinal İsmi   34886 non-null  object
 2   Yapıldığı Ülke  34886 non-null  object
 3   Yönetmeni       34886 non-null  object
 4   Oyuncu Kadrosu  33464 non-null  object
 5   Türü            34858 non-null  object
 6   Veri Kaynağı    34886 non-null  object
 7   Ozet            34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [46]:
df = df.drop(columns=['simgelestirilmis','sadelestirilmis','ozunuBulmus','Olay Dizisi'])
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Çıkış Yılı      34886 non-null  int64 
 1   Orijinal İsmi   34886 non-null  object
 2   Yapıldığı Ülke  34886 non-null  object
 3   Yönetmeni       34886 non-null  object
 4   Oyuncu Kadrosu  33464 non-null  object
 5   Türü            34858 non-null  object
 6   Veri Kaynağı    34886 non-null  object
 7   Ozet            34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


Arama yapmadan önce gerekli TF / IDF matrisini (dizey) oluşturalım.

**Kaynaklar:**

* https://monkeylearn.com/blog/what-is-tf-idf/
* https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a


In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords
aranilacak_tfidfNesnesi = TfidfVectorizer(stop_words=stopwords.words("turkish"))
aranilacak_tfidfDizeyi = aranilacak_tfidfNesnesi.fit_transform(df['Ozet'])
aranilacak_tfidfDizeyi.shape

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(34886, 87945)

ozBulucu TurkishStemmer olunca aranilacak_tfidfDizeyi.shape (34886, 113148)

ozBulucu nltk.stem olunca aranilacak_tfidfDizeyi.shape (34886, 215685)

ozBulucu ve veri ön işlemleri olmadan aranilacak_tfidfDizeyi.shape (34886, 225577)

ozBulucu Zemberek olunca aranilacak_tfidfDizeyi.shape (34886, 88404)


In [114]:
aranilacak_tfidfNesnesi.get_feature_names()[5000:5010]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['axelson',
 'axelstein',
 'axew',
 'axford',
 'axis',
 'axl',
 'axlerod',
 'axmann',
 'axton',
 'axwell']

In [49]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(aranilacak_tfidfDizeyi, aranilacak_tfidfDizeyi)
cosine_sim.shape

(34886, 34886)

Yukarıdaki hesabı yapması için yaklaşık 25 GB'lık RAM yeterli oldu.

In [115]:
indices = pd.Series(df.index, index=df['Orijinal İsmi']).drop_duplicates()
print(indices[:10])

Orijinal İsmi
Kansas Saloon Smashers              0
Love by the Light of the Moon       1
The Martyred Presidents             2
Terrible Teddy, the Grizzly King    3
Jack and the Beanstalk              4
Alice in Wonderland                 5
The Great Train Robbery             6
The Suburbanite                     7
The Little Train Robbery            8
The Night Before Christmas          9
dtype: int64


In [116]:
def tavsiyeVer(filmBasligi, cosine_sim=cosine_sim):
    idx = indices[filmBasligi]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['Orijinal İsmi'].iloc[movie_indices]

In [117]:
print(tavsiyeVer(gereksizKelimelerCikarilmisArananFilmOzeti))

KeyError: ignored